In [2]:
import pandas as pd

data = pd.read_csv('all_glucose_prediction_results.csv')
data

,Category,Start Date,Previous Lower Bound,Actual Glucose Change,Previous Upper Bound,Actual Variance,Prediction Status,Predicted Glucose Change,Predicted Variance
0,Lows,2023-10-02,0.295662,0.373529,0.427868,0.061912,True,0.361765,0.066103
1,Lows,2023-10-03,0.311618,0.439474,0.435441,0.101827,False,0.373529,0.061912
2,Lows,2023-10-04,0.337646,0.497619,0.541301,0.111619,True,0.439474,0.101827
3,Lows,2023-10-05,0.386000,0.507143,0.609238,0.105571,True,0.497619,0.111619
4,Lows,2023-10-06,0.401571,0.545238,0.612714,0.111476,True,0.507143,0.105571
...,...,...,...,...,...,...,...,...,...
374,Very Highs,2024-01-22,0.170126,0.796296,1.303207,0.611140,True,0.736667,0.566540
375,Very Highs,2024-01-24,0.185157,0.706818,1.407436,0.427045,True,0.796296,0.611140
376,Very Highs,2024-01-26,0.279773,0.720000,1.133864,0.408795,True,0.706818,0.427045
377,Very Highs,2024-01-30,0.311205,0.726087,1.128795,0.395193,True,0.720000,0.408795


In [3]:
category_stats = data.groupby('Category')[['Predicted Glucose Change', 'Predicted Variance', 'Actual Glucose Change', 'Actual Variance']].agg(
    ['mean', 'median']
).reset_index()

category_order = ["Lows", "Mediums", "Highs", "Very Highs"]

category_stats_ordered = category_stats.set_index('Category').loc[category_order].reset_index()

print("Category Statistics in Custom Order (Lows, Mediums, Highs, Very Highs):")
print(category_stats_ordered)

Category Statistics in Custom Order (Lows, Mediums, Highs, Very Highs):
     Category Predicted Glucose Change           Predicted Variance            \
                                  mean    median               mean    median   
0        Lows                 0.547615  0.536667           0.293084  0.224280   
1     Mediums                 0.620751  0.586207           0.319169  0.304200   
2       Highs                 0.602902  0.595833           0.311725  0.274281   
3  Very Highs                 0.664060  0.648214           0.290569  0.228806   

  Actual Glucose Change           Actual Variance            
                   mean    median            mean    median  
0              0.549556  0.543478        0.296464  0.227325  
1              0.622148  0.589286        0.317872  0.303023  
2              0.603857  0.595833        0.310909  0.274281  
3              0.665216  0.648214        0.290743  0.228806  


In [4]:
data_false_predictions = data[data['Prediction Status'] == False].copy()

if not data_false_predictions.empty:
    data_false_predictions.loc[:, 'Absolute Error Glucose Change'] = abs(data_false_predictions['Actual Glucose Change'] - data_false_predictions['Predicted Glucose Change'])
    data_false_predictions.loc[:, 'Absolute Error Variance Change'] = abs(data_false_predictions['Actual Variance'] - data_false_predictions['Predicted Variance'])

    mae_stats_glucose = data_false_predictions.groupby('Category')['Absolute Error Glucose Change'].agg(['mean', 'median']).reset_index()
    mae_stats_variance = data_false_predictions.groupby('Category')['Absolute Error Variance Change'].agg(['mean', 'median']).reset_index()

    available_categories_glucose = [cat for cat in category_order if cat in mae_stats_glucose['Category'].values]
    mae_stats_glucose_ordered = mae_stats_glucose.set_index('Category').loc[available_categories_glucose].reset_index()

    available_categories_variance = [cat for cat in category_order if cat in mae_stats_variance['Category'].values]
    mae_stats_variance_ordered = mae_stats_variance.set_index('Category').loc[available_categories_variance].reset_index()

    print("MAE Statistics for Glucose Change (False Predictions):")
    print(mae_stats_glucose_ordered)
    
    print("\nMAE Statistics for Variance Change (False Predictions):")
    print(mae_stats_variance_ordered)
else:
    print("No entries found with Prediction Status as False.")

MAE Statistics for Glucose Change (False Predictions):
     Category      mean    median
0        Lows  0.131186  0.131186
1       Highs  0.105769  0.105769
2  Very Highs  0.168010  0.168010

MAE Statistics for Variance Change (False Predictions):
     Category      mean    median
0        Lows  0.202372  0.202372
1       Highs  0.080235  0.080235
2  Very Highs  0.133928  0.133928
